In [1]:
import numpy as np
import glob
import scipy.io as sio
import torch
from torch import nn
import random
import csv
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import yaml
from evaluate import error
import time
import re
from syn_DI_dataset import make_dataset, make_dataloader

In [2]:
dataset_root = 'd:\Data\My_MMFi_Data\MMFi_Dataset'
# xian zai shi yong de shi Radar_Fused
with open('config_copy.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

['mmwave']
S02 ['A01', 'A02', 'A03', 'A04', 'A06', 'A08', 'A09', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A26']
S03 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A15', 'A16', 'A17', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S05 ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A19', 'A20', 'A21', 'A24', 'A25']
S06 ['A01', 'A03', 'A04', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A18', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S08 ['A01', 'A02', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A13', 'A14', 'A17', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27']
S09 ['A01', 'A07', 'A08', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A17', 'A18', 'A20', 'A21', 'A24', 'A25', 'A27']
S11 ['A01', 'A02', 'A03', 'A05', 'A06', 'A08', 'A10', 'A11', 'A12', 'A13', 'A16', 'A17', 'A18', 'A19', 'A

In [3]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.

    dict_keys(['modality', 'scene', 'subject', 'action', 'idx', 'output', 
    'input_rgb', 'input_depth', 'input_lidar', 'input_mmwave'])
    '''
    ## get sequence lengths
    # for t in batch:
    #     print(t.keys())
    #     print(a)
    # #     # print(t[0].shape,t[1].shape)
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))

    lengths = torch.tensor([t['input_mmwave'].shape[0] for t in batch ])

    # # rgb
    # rgb_data = np.array([(t['input_rgb']) for t in batch ])
    # rgb_data = torch.FloatTensor(rgb_data).permute(0,3,1,2)

    # # depth
    # depth_data = np.array([(t['input_depth']) for t in batch ])
    # depth_data = torch.FloatTensor(depth_data).permute(0,3,1,2)

    # mmwave
    ## padd
    mmwave_data = [torch.Tensor(t['input_mmwave']) for t in batch ]
    mmwave_data = torch.nn.utils.rnn.pad_sequence(mmwave_data)
    ## compute mask
    mmwave_data = mmwave_data.permute(1,0,2)

    # # lidar
    # ## padd
    # lidar_data = [torch.Tensor(t['input_lidar']) for t in batch ]
    # lidar_data = torch.nn.utils.rnn.pad_sequence(lidar_data)
    # ## compute mask
    # lidar_data = lidar_data.permute(1,0,2)

    # # wifi-csi
    # wifi_data = np.array([(t['input_wifi-csi']) for t in batch ])
    # wifi_data = torch.FloatTensor(wifi_data)
    # modality_list = random.choices(
    #     [True, False],
    #     k= 5,
    #     weights=[80, 20]
    # )

    # return rgb_data, depth_data, lidar_data, mmwave_data, wifi_data, kpts, modality_list
    return mmwave_data, kpts, lengths

In [4]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)

In [10]:
min_length = 100
for i, data in enumerate(tqdm(train_loader)):
    mmwave_data, kpts, length = data
    # print(mmwave_data.shape)
    if mmwave_data.shape[1] <= min_length:
        min_length = mmwave_data.shape[1]
        print(mmwave_data.shape)
    if mmwave_data.shape[1] <= 32:
        print(mmwave_data.shape)

  0%|          | 25/6495 [00:00<00:53, 121.18it/s]

torch.Size([32, 67, 5])
torch.Size([32, 57, 5])
torch.Size([32, 56, 5])


  1%|▏         | 90/6495 [00:00<00:51, 123.18it/s]

torch.Size([32, 55, 5])


  2%|▏         | 155/6495 [00:01<00:51, 122.65it/s]

...

 38%|███▊      | 2492/6495 [00:18<00:29, 135.71it/s]

torch.Size([32, 47, 5])


 51%|█████     | 3300/6495 [00:24<00:23, 137.02it/s]

torch.Size([32, 43, 5])


100%|██████████| 6495/6495 [00:48<00:00, 134.61it/s]
